In [ ]:
import numpy as np
import pandas as pd
import os
import time
import threading
import tkinter as tk
from tkinter import ttk
from binance import Client


In [ ]:
api_key = os.environ.get('API_KEY')
secret_key = os.environ.get('API_KEY_PRIVATE')

# Create connection to Binance
client = Client(api_key, secret_key)

In [ ]:
#read BTCUSDT data file
df = pd.read_csv('data/BTCUSDT_data.csv')
#drop unamed and index columns
df.drop(["Unnamed: 0", "index"], axis = 1, inplace = True)

In [ ]:
def sma(close, period):
    ma = close.rolling(period).mean()
    return pd.Series(ma)


In [ ]:
def ema(close, period):
    """ This function takes closing price and period and returns exponential moving average"""
    close = close.squeeze()
    #calculate Multiplier (smoothing factor)
    multiplier = 2 / (period + 1)
    #calculate Simple Moving Average
    ma = sma(close, period)
    #initialize the Exponential Moving Average list
    ema = [np.NaN] * len(close)

    #iterate over the closing prices
    for index in range(len(close)):
        #check index equals the period calculate EMA
        if index == period:
            ema[index] = close.iloc[index] * multiplier + ma[index - 1] * (1 - multiplier)
        #check index is greater than the period calculate follwing EMA
        if index > period:
            ema[index] = close.iloc[index] * multiplier + ema[index - 1] * (1 - multiplier)

    return pd.Series(ema)

In [ ]:
def macd(close, fast_period = 12, slow_period = 26, smoothing = 9):
    """ This function takes in closing price, a period of 12 and 26 and a smoothing value and returns"""

    #calculate MACD using the difference of EMA periods 12 and 26
    macd = ema(close, 12) - ema(close, 26)
    #create series for signal calculation
    empty_values = pd.Series([np.NaN]*slow_period)

    #calculate signal line using smoothing value
    signal_calc_values = macd.iloc[slow_period::].reset_index()
    signal_calc_values.drop(['index'], axis = 1, inplace = True)
    signal = pd.concat([empty_values, ema(signal_calc_values[0], smoothing)])
    signal = pd.Series(signal.reset_index().drop(['index'], axis = 1)[0])

    #calculate histogram
    histogram = macd - signal

    return macd, signal, histogram

    

In [ ]:
def ichimoku_cloud(high, low, close, conversion_period = 9, base_period = 26, leading_period = 52):
    """ This function takes in price data for high, low, close and also conversion, base and leading periods 
    returning conversion and base line also leading and lagging spans"""
    #calculate conversion line using high and low prices of last 9 periods
    conversion_line = (high.rolling(conversion_period).max() + low.rolling(conversion_period).min()) / 2
    #calculate base line using high and low prices of last 26 periods
    base_line = (high.rolling(base_period).max() + low.rolling(base_period).min()) / 2

    #calculate leading span using high and low prices using period of 52
    leading_span_a = ((conversion_line + base_line) / 2).shift(base_period)
    leading_span_b = (high.rolling(leading_period).max() + low.rolling(leading_period).min()) / 2
    leading_span_b = leading_span_b.shift(base_period)

    #calculate lagging span by shifting using base period
    lagging_span = close.shift(-base_period)
    
    return conversion_line, base_line, leading_span_a, leading_span_b, lagging_span
    

In [ ]:
def calculate_atr(data, period = 14):
    """calculate ATR using high, low, close data and period"""
    
    #separate price data
    high = data['High']
    low = data['Low']
    close = data['Close']

    #create empty series
    atr = pd.Series(0.0, index=data.index)
    #set ATR to closing value
    atr[0] = close[0]

    #for each remaining close data calculate ATR
    for i in range(1, len(close)):
        
        range1 = high[i] - low[i]
        range2 = abs(high[i] - close[i-1])
        range3 = abs(low[i] - close[i-1])
        #using the difference of prices true range takes max value of ranges
        true_range = max(range1, range2, range3)
        atr[i] = ((period - 1) * atr[i-1] + true_range) / period

    return atr

In [ ]:
# def calculate_obv(data):
#     close = data['Close']
#     volume = data['Volume']
#     obv = pd.Series(0, index=data.index)
#     obv[0] = volume[0]
#     for i in range(1, len(close)):
#         if close[i] > close[i-1]:
#             obv[i] = obv[i-1] + volume[i]
#         elif close[i] < close[i-1]:
#             obv[i] = obv[i-1] - volume[i]
#         else:
#             obv[i] = obv[i-1]
#     return obv

In [ ]:
def calculate_donchian_channels(data, window_size = 20):
    """calculate donchain using closing data and n value of 20 parameters
    returns upper and lower channels and midline"""

    #separate price data
    close = data['Close']

    #calculate upperchannel using max value in window
    upper_channel = close.rolling(window=window_size).max()
    #calculate lowerchannel using min value in window
    lower_channel = close.rolling(window=window_size).min()
    #calculate midline using upper and lower channel /2
    mid_line = (upper_channel + lower_channel) / 2
    
    return upper_channel, lower_channel, mid_line


In [ ]:
import numpy as np
import pandas as pd

def rsi(close, number_of_periods=14):
    """ calculate RSI using closing price and number of periods
    returns RSI"""

    close = pd.Series(close.squeeze())  # Ensure input is a pandas Series

    deltas = close.diff()
    up, down = deltas.copy(), deltas.copy()

    up[up < 0] = 0
    down[down > 0] = 0
    down = down.abs()

    avg_gain = up.rolling(window=number_of_periods, min_periods=number_of_periods).mean()
    avg_loss = down.rolling(window=number_of_periods, min_periods=number_of_periods).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi.values


In [ ]:
def bollinger_bands(close, window_size=20):
    """calculate bollinger bands using closing price parameters closing price, window size
    returns upper, lower and rolling mean for bollinger bands"""
    #calculate rolling mean of closing price
    rolling_mean = close.rolling(window_size).mean()
    #calculate standard deviation of closing price
    rolling_std = close.rolling(window_size).std()

    #calculate upper band using rolling mean and rolling standard deviation
    upper_band = rolling_mean + (rolling_std * 2)
    #calculate lower band using rolling mean and rolling standard deviation
    lower_band = rolling_mean - (rolling_std * 2)

    return upper_band, rolling_mean, lower_band

In [ ]:
def fibonacci_retracement(close, open):
    """calculate fibonacci retracement using close and open price
    returns fibonacci retracement in separate dataframe"""
    #fibonacci levels
    fib_levels = [0, 0.236, 0.382, 0.5, 0.618, 0.764, 1.0]

    diff = close - open
    #list comprehension
    levels = np.array([close - level * diff for level in fib_levels])
    df = pd.DataFrame(levels.T, columns=[f"Fibonacci {int(level*100)}%" for level in fib_levels])
    return df

In [ ]:
def use_macd(df):
    df['macd'], df['signal'], df['histogram'] = macd(df.Close)
    df['prev_histogram'] = df['histogram'].shift(1)
    # trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    df['buy_sell'] = 0

    buy_condition_1 = (df['prev_histogram'] < 0) & (df['histogram'] > 0) 
    buy_condition_2 = (df['macd'] < 0) & (df['signal'] < 0)
    
    df.loc[buy_condition_1 & buy_condition_2, 'buy_sell'] = 1

    sell_signal_1 = (df['prev_histogram'] > 0) & (df['histogram'] < 0) 
    sell_signal_2 = (df['macd'] > 0) & (df['signal'] > 0)

    df.loc[sell_signal_1 & sell_signal_2, 'buy_sell'] = -1

    #Clean up df
    df.drop(['macd', 'signal', 'histogram', 'prev_histogram'], axis = 1, inplace = True)

    return df

In [ ]:
# Testing macd_function
macd_test = use_macd(df)
macd_test[(macd_test['buy_sell'] == 1) | (macd_test['buy_sell'] == -1)]

In [ ]:
def use_sma(df):
    df['sma_50'] = sma(df.Close, 50)
    df['sma_200'] = sma(df.Close, 200)
    df['sma_diff'] = df['sma_50'] - df['sma_200']
    df['prev_sma_diff'] = df['sma_diff'].shift(1)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = (trading_df['sma_diff'] > 0) & (trading_df['prev_sma_diff'] < 0)
    sell_condition = (trading_df['sma_diff'] < 0) & (trading_df['prev_sma_diff'] > 0)

    trading_df.loc[buy_condition, 'buy_sell'] = 1
    trading_df.loc[sell_condition, 'buy_sell'] = -1

    # Clean up df
    df.drop(['sma_50', 'sma_200', 'sma_diff', 'prev_sma_diff'], axis = 1, inplace = True)

    return trading_df

In [ ]:
# Test sma
sma_test = use_sma(df)
sma_test[(sma_test['buy_sell'] == 1) | (sma_test['buy_sell'] == -1)]

In [ ]:
def use_rsi(df, overbought_thresh = 70, oversold_thresh = 30):
    df['rsi'] = rsi(df.Close)
    df['prev_rsi'] = df['rsi'].shift(1)

    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = (trading_df['rsi'] < oversold_thresh) & (trading_df['prev_rsi'] >= oversold_thresh)
    sell_condition = (trading_df['rsi'] > overbought_thresh) & (trading_df['prev_rsi'] <= overbought_thresh)

    trading_df.loc[buy_condition, 'buy_sell'] = 1
    trading_df.loc[sell_condition, 'buy_sell'] = -1
    # Clean up df
    df.drop(['rsi', 'prev_rsi'], axis = 1, inplace = True)
    return trading_df

In [ ]:
# Test rsi
rsi_test = use_rsi(df)
rsi_test[(rsi_test['buy_sell'] == 1) | (rsi_test['buy_sell'] == -1)]

In [ ]:
def use_ichimoku(df):
    df['conversion_line'], df['base_line'], df['leading_span_a'], df['leading_span_b'], df['lagging_span'] = ichimoku_cloud(df.High, df.Low, df.Close)
    df['conversion_base_diff'] = df['conversion_line'] - df['base_line']
    df['prev_diff'] = df['conversion_base_diff'].shift(1)
    
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition_1 = trading_df.Close >= np.maximum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    buy_condition_2 = (trading_df['conversion_base_diff'] > 0) & (trading_df['prev_diff'] < 0)
    trading_df.loc[buy_condition_1 & buy_condition_2, 'buy_sell'] = 1

    sell_condition_1 = trading_df.Close < np.minimum(trading_df['leading_span_a'], trading_df['leading_span_b'])
    sell_condition_2 = (trading_df['conversion_base_diff'] < 0) & (trading_df['prev_diff'] > 0)
    trading_df.loc[sell_condition_1 & sell_condition_2, 'buy_sell'] = -1

    # Clean up df
    df.drop(['conversion_line', 'base_line', 'leading_span_a', 'leading_span_b', 'lagging_span', 'conversion_base_diff', 'prev_diff'], axis = 1, inplace = True)
    return trading_df

In [ ]:
#Test ichimoku
ichimoku_test = use_ichimoku(df)
ichimoku_test[(ichimoku_test['buy_sell'] == 1) | (ichimoku_test['buy_sell'] == -1)]

In [ ]:
def use_donchian_channel(df):
    df['upper'], df['lower'], df['mid'] = calculate_donchian_channels(df)
    trading_df = df[(df.Time >= '2022-12-01 00:00:00') & (df.Time <= '2023-02-01 23:59:00')]
    trading_df['buy_sell'] = 0

    buy_condition = trading_df.Close >= trading_df.upper
    trading_df.loc[buy_condition, 'buy_sell'] = 1

    sell_condition = trading_df.Close <= trading_df.lower
    trading_df.loc[sell_condition, 'buy_sell'] = -1
    
    df.drop(['upper', 'lower', 'mid'], axis = 1, inplace = True)
    return trading_df
    

In [ ]:
donchian_test = use_donchian_channel(df)
donchian_test[(donchian_test['buy_sell'] == 1) | (donchian_test['buy_sell'] == -1)]


In [ ]:
def get_data(symbol, interval, lookback):
    data = []
    while data == []:
        data = client.get_historical_klines(symbol,
                                      interval,
                                      str(lookback) + ' minute ago UTC')
    df = pd.DataFrame(data)
    df = df.iloc[:,:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.insert(loc=0, column='Name', value=symbol)
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    df.Time = pd.to_datetime(df.Time, unit = 'ms')
    # df.set_index('Time', inplace = True)
    
    return df

In [ ]:
def get_ta(df, indicator = macd):
    df = use_macd(df)
    return df

In [ ]:
def buy(trading_data, buy_type, fiat_amount):  
    trading_data = trading_data[(trading_data['buy_sell'] == 1) | (trading_data['buy_sell'] == -1)]
    if buy_type == 'all':
        position = False
        coin = 0
        for idx, row in trading_data.iterrows():
            if (row['buy_sell'] == 1) & (position == False):
                position = True
                coin = fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0
                
            elif (row['buy_sell'] == -1) & (position == True):
                fiat_amount = coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    
    if buy_type == 'consec':
        position = False
        fiat_buy = fiat_amount / 4
        coin = 0
        for idx, row in trading_data.iterrows():
            if (row['buy_sell'] == 1) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif (row['buy_sell'] == 1) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / row['Close']
                #print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif (row['buy_sell'] == -1) & (position == True):
                fiat_amount += coin * row['Close']
                #print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
        fiat_amount += coin * trading_data.iloc[-1, 5]
        coin = 0
    return fiat_amount

In [ ]:
def backtest(indicator, data = df, buy_type='all', fiat_amount = 0 ):
    # while True:
        # try:
        #     fiat_amount = float(input("Please enter your fiat value: "))
        # except ValueError:
        #     #user input not valid
        #     print("please enter a valid number")
        #     continue
        # else:
    if indicator == 'macd': 
        trading_data = use_macd(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)
    
    elif indicator == 'rsi':
        trading_data = use_rsi(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)

    elif (indicator == 'ma') | (indicator == 'sma'):
        trading_data = use_sma(df)
        fiat_amount = buy(trading_data, buy_type=buy_type, fiat_amount=fiat_amount)    

    return fiat_amount
        #successful input break loop and continue
    # break

    

In [ ]:
def trade(token, indicator,stop_event, fiat_amount = 0,  buy_type = 'all'):
    df = get_data(token, Client.KLINE_INTERVAL_1MINUTE, '500')
    df = get_ta(df)
    time.sleep(60)
    if buy_type == 'all':
        position = False
        coin = 0
        while not stop_event.is_set():
            file_mod_time = os.stat('output.txt').st_mtime
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            if (df.iloc[-1,-1] == 1) & (position == False):
                position = True
                coin = fiat_amount / df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Bought {} with {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                fiat_amount = 0
                
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount = coin * df.iloc[-1, 5]
                with open("output.txt", "a+") as file:
                    file.write("Sold {} for {}".format(coin, fiat_amount) +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
                coin = 0
                position = False
            elif (position == False):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Buy signal..." +"\n" )
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            elif (position == True):
                with open("output.txt", "a+") as file:
                    file.write("Waiting for Sell signal..." +"\n")
                    file.write("File modified after {}".format(time.time() - file_mod_time))
            time.sleep(60)

    if buy_type == 'consec':
        position = False
        coin = 0
        while not stop_event.is_set():
            df = pd.concat([df, get_data('BTCUSDT', Client.KLINE_INTERVAL_1MINUTE, '1')], ignore_index=True)
            df = get_ta(df)
            fiat_buy = fiat_amount / 4
            if (df.iloc[-1,-1] == 1) & (fiat_amount > fiat_buy):
                position = True
                coin += fiat_buy / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount -= fiat_buy
            elif (df.iloc[-1,-1] == 1) & (fiat_amount < fiat_buy) & (fiat_amount > 0):
                position = True
                coin += fiat_amount / df.iloc[-1, 5]
                print("Bought {} with {}".format(coin, fiat_amount))
                fiat_amount = 0 
            elif (df.iloc[-1,-1] == -1) & (position == True):
                fiat_amount += coin * df.iloc[-1, 5]
                print("Sold {} for {}".format(coin, fiat_amount))
                coin = 0
                fiat_buy = fiat_amount / 4
                position = False
            time.sleep(60)

   

In [ ]:
#trade(indicator = 'sma', buy_type='consec')

In [ ]:
result = {'BTC': [],
               'ETH': [],
               'APE': [],
               'CHZ': [],
               'DOG': [],
               'LIN': [],
               'MAN': [],
               'UNI': [],}

files = ['BTCUSDT_data.csv', 'ETHUSDT_data.csv', 'APEUSDT_data.csv', 'CHZUSDT_data.csv', 'DOGEUSDT_data.csv', 'LINKUSDT_data.csv', 'MANAUSDT_data.csv', 'UNIUSDT_data.csv']

#for file in files:
#    df = pd.read_csv('data/'+file)
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='all'))
#    result[file[0:3]].append(trade(indicator = 'rsi', buy_type='consec'))
    

In [ ]:
#result_macd

Gerald's User Input Prompt to Select the two TA

In [ ]:
indicator_functions = {
    'EMA': lambda data: ema(data, 14),
    'MACD': macd,
    'Ichimoku Cloud': ichimoku_cloud,
    'ATR': calculate_atr,
    'Donchian Channels': calculate_donchian_channels,
    'RSI': rsi,
    'Bollinger Bands': bollinger_bands,
    # 'Fibonacci Retracement': fibonacci_retracement,
}

token_list = {
    'BTC': 'BTCUSDT',
    'ETH': 'ETHUSDT',
    'APE': 'APEUSDT',
    'CHZ': 'CHZUSDT',
    'DOG': 'DOGEUSDT',
    'LIN': 'LINKUSDT',
    'MAN': 'MANAUSDT',
    'UNI': 'UNIUSDT',
}


tokens = list(token_list.keys())

def live_trade():
    indicator1 = selected_indicator1.get()
    # indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    with open("output.txt", "w") as file:
            file.write("Initializing..." + "\n")
    
    stop_event = threading.Event()
    t = threading.Thread(target=trade, args=(token_list[token], indicator1.lower(),stop_event, fiat_value, 'all'))
    print("Start Threading")
    t.start()
    new_window = tk.Tk()
    new_window.title("Trading Process")
    
    output_text = tk.Text(new_window)
    output_text.insert(tk.END, 'Initializing...')
    output_text.pack()
    output_text.config(state=tk.DISABLED)
    # trade(token_list[token], indicator=indicator1.lower(), fiat_amount=fiat_value)

    #Multithreading
    update_tk(output_text,new_window)
    new_window.protocol("WM_DELETE_WINDOW", lambda: stop_worker(stop_event,new_window))

    
    new_window.mainloop()

data_files = {
    'BTC': 'data/BTCUSDT_data.csv',
    'ETH': 'data/ETHUSDT_data.csv',
    'APE': 'data/APEUSDT_data.csv',
    'CHZ': 'data/CHZUSDT_data.csv',
    'DOG': 'data/DOGEUSDT_data.csv',
    'LIN': 'data/LINKUSDT_data.csv',
    'MAN': 'data/MANAUSDT_data.csv',
    'UNI': 'data/UNIUSDT_data.csv',
}

dataframes = {}
for token, file in data_files.items():
    df = pd.read_csv(file)
    df.drop(["Unnamed: 0", "index"], axis=1, inplace=True)
    dataframes[token] = df

def submit():
    indicator1 = selected_indicator1.get()
    indicator2 = selected_indicator2.get()
    fiat_value = float(fiat_value_entry.get())
    token = selected_token.get()

    if not token:
        print("Error: Please select a token.")
        return
    window.destroy()
    new_window = tk.Tk()
    new_window.title("Trade Result")
    data = dataframes[token]
    # Calculate the outcomes for the selected indicators
    outcome1 = backtest(indicator=indicator1.lower(), data=data, fiat_amount=fiat_value)
    outcome2 = backtest(indicator=indicator2.lower(), data=data, fiat_amount=fiat_value)
    output_text = tk.Text(new_window, height=10, width=50)
    output_text.pack()
    output_text.insert(tk.END, f"Selected Indicators:\n")
    output_text.insert(tk.END, f"Indicator 1: {indicator1}, Outcome: {outcome1}\n")
    output_text.insert(tk.END, f"Indicator 2: {indicator2}, Outcome: {outcome2}\n")
    output_text.insert(tk.END, f"Fiat Value: {fiat_value}, Token: {token}\n")


def update_tk(output_text, window):
    with open("output.txt", "r") as file:
        content = file.read()
    output_text.config(state=tk.NORMAL)
    output_text.delete('1.0', tk.END)
    output_text.insert(tk.END, content)
    output_text.config(state=tk.DISABLED) 
    print('Screen refreshed')
    window.after(61500, lambda: update_tk(output_text, window))

def stop_worker(stop_event, window):
    stop_event.set()
    window.destroy()





# Create the tkinter window
window = tk.Tk()
window.title("Select Technical Analysis Indicators, Fiat Value, and Token")

# Create the labels and dropdowns for the two indicators
label1 = ttk.Label(window, text="Select Indicator 1:")
label1.grid(column=0, row=0)
selected_indicator1 = tk.StringVar()
dropdown1 = ttk.Combobox(window, textvariable=selected_indicator1, values=list(indicator_functions.keys()))
dropdown1.grid(column=1, row=0)

label2 = ttk.Label(window, text="Select Indicator 2 (Backtest only):")
label2.grid(column=0, row=1)
selected_indicator2 = tk.StringVar()
dropdown2 = ttk.Combobox(window, textvariable=selected_indicator2, values=list(indicator_functions.keys()))
dropdown2.grid(column=1, row=1)

# Create the labels and entry for the fiat value
label3 = ttk.Label(window, text="Enter Fiat Value:")
label3.grid(column=0, row=2)
fiat_value_entry = ttk.Entry(window)
fiat_value_entry.grid(column=1, row=2)

#Create the labels and dropdown for the token
label4 = ttk.Label(window, text="Select Token:")
label4.grid(column=0, row=3)
selected_token = tk.StringVar()
dropdown3 = ttk.Combobox(window, textvariable=selected_token, values=tokens)
dropdown3.grid(column=1, row=3)

#Create a submit button
submit_button = ttk.Button(window, text="Trade", command=live_trade)
submit_button.grid(column=1, row=4)
submit_button = ttk.Button(window, text="Backtest", command=submit)
submit_button.grid(column=0, row=4)

# Start the tkinter main loop
window.mainloop()


# Backtest

In [ ]:
df = pd.read_csv('./year_data/ETHUSDT_full.csv')

In [ ]:
files = ['BTCUSDT_full.csv', 'ETHUSDT_full.csv', 'APEUSDT_full.csv', 'CHZUSDT_full.csv', 'DOGEUSDT_full.csv', 'LINKUSDT_full.csv', 'MANAUSDT_full.csv', 'UNIUSDT_full.csv']
indicators = [use_macd, use_rsi, use_sma, use_donchian_channel, use_ichimoku]
result_df = pd.DataFrame(index=['BTC','ETH','APE','CHZ','DOG','LIN','MAN','UNI'],columns=['use_macd & use_rsi', 'use_macd & use_sma', 'use_rsi & use_sma'])
for file in files:
    df = pd.read_csv("./year_data/{}".format(file))
    for i in range(len(indicators)):
        for j in range(i + 1,len(indicators)):
            print('Testing:{} & {}'.format(indicators[i].__name__, indicators[j].__name__))
            trading_frame_1 = indicators[i](df)
            trading_frame_2 = indicators[j](df)
            trading_data = df.copy()
            trading_data['buy_sell_1'] = trading_frame_1.iloc[:,-1]
            trading_data['buy_sell_2'] = trading_frame_2.iloc[:,-1]
            
            position = False
            coin = 0
            fiat_amount = 1000
            for idx, row in trading_data.iterrows():
                if (row['buy_sell_1'] == 1) & (row['buy_sell_2'] == 1) & (position == False):
                    position = True
                    coin = fiat_amount / row['Close']
                    print("Bought {} with {}".format(coin, fiat_amount))
                    fiat_amount = 0
                    
                elif ((row['buy_sell_1'] == -1) | (row['buy_sell_2'] == -1)) & (position == True):
                    fiat_amount = coin * row['Close']
                    print("Sold {} for {}".format(coin, fiat_amount))
                    coin = 0
                    position = False
            fiat_amount += coin * trading_data.iloc[-1, 5]
            coin = 0
            
            result_df.loc[file[0:3], '{} & {}'.format(indicators[i].__name__, indicators[j].__name__) ] = fiat_amount



In [ ]:
result_df